In [ ]:
import numpy as np
import json
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

In [ ]:
vgg16 = VGG16(weights='imagenet')

In [ ]:
vgg16.save('vgg16.h5')

In [ ]:
def prepare_image (img_path):
  img = load_img(img_path, target_size=(224,224))
  x = img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x=preprocess_input(x)
  return x

In [ ]:
y=prepare_image('0006.jpg')

In [ ]:
preds=vgg16.predict(y)

1/1 [==============================] - 1s 788ms/step


In [ ]:
preds.shape

(1, 1000)

In [ ]:
CLASS_INDEX = None

In [ ]:
CLASS_INDEX_PATH = 'imagenet_class_index.json'

In [ ]:
def get_predictions(preds,top=5):
  global CLASS_INDEX
  CLASS_INDEX=json.load(open(CLASS_INDEX_PATH))
  results=[]
  for pred in preds:
    top_indices = pred.argsort()[-top:][::-1]
    result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],)for i in top_indices]
    result.sort(key=lambda x: x[2],reverse=True)
    results.append(result)
  return results

In [ ]:
print(get_predictions(preds,top=5))

[[('n02814533', 'beach_wagon', 0.6324493), ('n03770679', 'minivan', 0.17383286), ('n02974003', 'car_wheel', 0.043032356), ('n03100240', 'convertible', 0.033382528), ('n03459775', 'grille', 0.02717272)]]


In [ ]:
from collections import Counter, defaultdict
import os
import pickle as pk

In [ ]:
def get_car_categories ():
  d = defaultdict(float)
  img_list = os.listdir('training/01-whole')
  for i, img_path in enumerate (img_list):
    img = prepare_image('training/01-whole/'+img_path)
    out = vgg16.predict(img)
    top = get_predictions(out, top=5)
    for j in top[0]:
      d[j[0:2]] += j[2]
    if i %100 == 0:
      print(i, '/', len(img_list), 'complete')
    return Counter(d)

In [ ]:
cat_counter=get_car_categories()

1/1 [==============================] - 1s 741ms/step
0 / 920 complete


In [ ]:
cat_counter

Counter({('n03770679', 'minivan'): 0.8418725728988647,
         ('n03930630', 'pickup'): 0.0664503201842308,
         ('n04461696', 'tow_truck'): 0.015209113247692585,
         ('n02814533', 'beach_wagon'): 0.013734405860304832,
         ('n03594945', 'jeep'): 0.011652226559817791})

In [ ]:
with open('cat_counter.pk','wb') as f:
  pk.dump(cat_counter,f,-1)

In [ ]:
with open ('cat_counter.pk', 'rb') as f:
  cat_counter = pk.load(f)

In [ ]:
cat_list = [k for k, v in cat_counter.most_common()[:27]]

In [ ]:
cat_list

[('n03770679', 'minivan'),
 ('n03930630', 'pickup'),
 ('n04461696', 'tow_truck'),
 ('n02814533', 'beach_wagon'),
 ('n03594945', 'jeep')]

In [ ]:
def car_categories_gate(image_path, cat_list):
  img = prepare_image(image_path)
  out = vgg16.predict(img)
  top = get_predictions(out,top=5)
  print ("Validating that this is a picture of your car...")
  for j in top[0]:
    if j[0:2] in cat_list:
      print (j[0:2])
      return "Validation complete - proceed to damage evaluation"
  return "Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting)"


In [ ]:
car_categories_gate('toy.webp', cat_list)

1/1 [==============================] - 1s 703ms/step
Validating that this is a picture of your car...


'Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting)'

In [ ]:
import os
import glob
import datetime

from keras.applications.vgg16 import VGG16, preprocess_input
import keras.utils as image
from keras.models import Model

from sklearn.preprocessing import LabelEncoder
import numpy as np
import h5py
import json
import time

In [ ]:
config={
  "model"           : "vgg16",
  "weights"         : "imagenet",
  "include_top"     : False,

  "train_path"      : "training",
  "test_path"       : "validation",
  "features_path"   : "features.h5",
  "labels_path"     : "labels.h5",
  "results"         : "results.txt",
  "classifier_path" : "classifier.pickle",
  "model_path"      : "model",

  "test_size"       : 0.20,
  "seed"            : 9,
  "num_classes"     : 2
}

In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
model_name    = config["model"]
weights     = config["weights"]
include_top   = config["include_top"]
train_path    = config["train_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
results     = config["results"]
model_path    = config["model_path"]

In [ ]:
if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(base_model.input, base_model.get_layer('fc1').output)
    image_size = (224, 224)
else:
    base_model = None

print ("Successfully loaded base model and model...")

Successfully loaded base model and model...


In [ ]:
# path to training dataset
train_labels = os.listdir(train_path)

# encode the labels
print ("Encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])

# variables to hold features and labels
features = []
labels   = []

# loop over all the labels in the folder
count = 1
for i, label in enumerate(train_labels):
    cur_path = train_path + "/" + label
    count = 1
    for image_path in glob.glob(cur_path + "/*.jpg"):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
        labels.append(label)
        print ("Processed - " + str(count))
        count += 1
    print ("Completed label - " + label)

# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print ("Training labels: {}".format(le_labels))
print ("Training labels shape: {}".format(le_labels.shape))

Encoding labels...
Completed label - 00-damage
1/1 [==============================] - 1s 756ms/step
Processed - 1
1/1 [==============================] - 1s 669ms/step
Processed - 2
1/1 [==============================] - 1s 666ms/step
Processed - 3
1/1 [==============================] - 1s 588ms/step
Processed - 4
1/1 [==============================] - 1s 667ms/step
Processed - 5
1/1 [==============================] - 1s 626ms/step
Processed - 6
1/1 [==============================] - 1s 596ms/step
Processed - 7
1/1 [==============================] - 1s 688ms/step
Processed - 8
1/1 [==============================] - 1s 659ms/step
Processed - 9
1/1 [==============================] - 1s 587ms/step
Processed - 10
1/1 [==============================] - 1s 600ms/step
Processed - 11
1/1 [==============================] - 1s 676ms/step
Processed - 12
1/1 [==============================] - 1s 635ms/step
Processed - 13
1/1 [==============================] - 1s 626ms/step
Processed - 14
1/1 [=====